In [2]:
#importing libraries to connect to MySql 
from sqlalchemy import create_engine as ce, text
import pandas as pd

In [3]:
#creating the dbserver engine and required MySQL queries to fetch the data
from urllib.parse import quote_plus
mysql_engine = ce("mysql+pymysql://root:%s@localhost:3306/dublinbikes"% quote_plus("aarya@1997"))
station_query = 'SELECT * FROM station_details'
api_data_query = 'SELECT * FROM apidata'
weather_data_query = 'SELECT * FROM weather_data'

In [4]:
#loading data into dataframes 

station_df = pd.read_sql_query(sql=text(station_query), con=mysql_engine.connect())
availability_df = pd.read_sql_query(sql=text(api_data_query), con=mysql_engine.connect())
weather_df = pd.read_sql_query(sql=text(weather_data_query), con=mysql_engine.connect())

display(station_df.head())
display(availability_df.head())
display(weather_df.head())

,number,name,address,latitude,longitude
0,42,SMITHFIELD NORTH,Smithfield North,53.349562,-6.278198
1,30,PARNELL SQUARE NORTH,Parnell Square North,53.353742,-6.265301
2,54,CLONMEL STREET,Clonmel Street,53.336021,-6.262980
3,108,AVONDALE ROAD,Avondale Road,53.359405,-6.276142
4,20,JAMES STREET EAST,James Street East,53.336597,-6.248109


,id,number,banking,bonus,bike_stands,available_bike_stands,available_bikes,status,last_update,weather,data_fetch_time
0,2169,42,0,0,30,17,13,OPEN,1677844344000,"{""id"": 803, ""icon"": ""04d"", ""main"": ""Clouds"", ""...",2023-03-03 11:59:23
1,2170,30,0,0,20,20,0,OPEN,1677844604000,"{""id"": 803, ""icon"": ""04d"", ""main"": ""Clouds"", ""...",2023-03-03 11:59:23
2,2171,54,0,0,33,20,13,OPEN,1677844381000,"{""id"": 803, ""icon"": ""04d"", ""main"": ""Clouds"", ""...",2023-03-03 11:59:23
3,2172,108,0,0,35,17,18,OPEN,1677844516000,"{""id"": 803, ""icon"": ""04d"", ""main"": ""Clouds"", ""...",2023-03-03 11:59:23
4,2173,20,0,0,30,13,16,OPEN,1677844191000,"{""id"": 803, ""icon"": ""04d"", ""main"": ""Clouds"", ""...",2023-03-03 11:59:23


,id,main,description,temp,feels_like,temp_min,temp_max,pressure,humidity,latitude,longitude,date_fetch_time
0,552,Clouds,broken clouds,279.98,276.50,279.51,280.36,1032.0,84.0,53.35,-6.28,2023-03-03 11:58:59
1,553,Clouds,broken clouds,280.01,276.54,279.53,280.39,1032.0,83.0,53.35,-6.27,2023-03-03 11:58:59
2,554,Clouds,broken clouds,279.95,276.46,279.46,280.32,1032.0,84.0,53.34,-6.26,2023-03-03 11:58:59
3,555,Clouds,broken clouds,279.87,276.36,279.42,280.27,1032.0,84.0,53.36,-6.28,2023-03-03 11:59:00
4,556,Clouds,broken clouds,279.97,276.49,279.49,280.34,1032.0,84.0,53.34,-6.25,2023-03-03 11:59:00


In [4]:
import json
#Dropping unnecessary columns 
availability_df = availability_df.drop(['banking','bonus','status','weather'], axis=1)

In [5]:
#Cleaning up weather_df 
def kelvin_to_celsius(temp):
    temp_val = int(temp)
    temp_val = temp_val - 273
    return temp_val
weather_df['temp'] = weather_df['temp'].apply(kelvin_to_celsius)
weather_df['feels_like'] = weather_df['feels_like'].apply(kelvin_to_celsius)
weather_df['temp_min'] = weather_df['temp_min'].apply(kelvin_to_celsius)
weather_df['temp_max'] = weather_df['temp_max'].apply(kelvin_to_celsius)

weather_df = weather_df.drop(['latitude','longitude','date_fetch_time'], axis=1)

weather_df

,id,main,description,temp,feels_like,temp_min,temp_max,pressure,humidity
0,552,Clouds,broken clouds,6,3,6,7,1032.0,84.0
1,553,Clouds,broken clouds,7,3,6,7,1032.0,83.0
2,554,Clouds,broken clouds,6,3,6,7,1032.0,84.0
3,555,Clouds,broken clouds,6,3,6,7,1032.0,84.0
4,556,Clouds,broken clouds,6,3,6,7,1032.0,84.0
...,...,...,...,...,...,...,...,...,...
1010789,1011341,Rain,light rain,13,13,12,14,1015.0,84.0
1010790,1011342,Clouds,broken clouds,13,13,12,14,1014.0,84.0
1010791,1011343,Clouds,broken clouds,13,13,12,14,1014.0,84.0
1010792,1011344,Clouds,broken clouds,13,13,12,14,1015.0,84.0


In [6]:
#Merging availability_df and weather_df using left join to get more weather related data
availability_df = pd.merge(availability_df, weather_df, on='id', how='left')

availability_df

,id,number,bike_stands,available_bike_stands,available_bikes,last_update,data_fetch_time,main,description,temp,feels_like,temp_min,temp_max,pressure,humidity
0,2169,42,30,17,13,1677844344000,2023-03-03 11:59:23,Clouds,broken clouds,6,3,5,7,1032.0,81.0
1,2170,30,20,20,0,1677844604000,2023-03-03 11:59:23,Clouds,broken clouds,6,3,6,7,1031.0,81.0
2,2171,54,33,20,13,1677844381000,2023-03-03 11:59:23,Clouds,broken clouds,6,3,5,7,1032.0,81.0
3,2172,108,35,17,18,1677844516000,2023-03-03 11:59:23,Clouds,broken clouds,6,3,6,7,1031.0,81.0
4,2173,20,30,13,16,1677844191000,2023-03-03 11:59:23,Clouds,broken clouds,6,3,6,7,1031.0,81.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1006948,1009117,39,20,19,1,1680685493000,2023-04-05 09:08:09,Clouds,broken clouds,12,11,11,12,1015.0,88.0
1006949,1009118,83,40,20,19,1680685591000,2023-04-05 09:08:09,Clouds,broken clouds,12,11,11,12,1015.0,87.0
1006950,1009119,92,40,32,8,1680685305000,2023-04-05 09:08:09,Clouds,broken clouds,12,11,11,12,1015.0,88.0
1006951,1009120,21,30,16,14,1680685561000,2023-04-05 09:08:09,Clouds,broken clouds,12,11,11,12,1015.0,88.0


In [7]:
#We are creating a model for each station where prediction will be done only if a station id is given as input. 
station_ids = station_df['number'].unique()
station_grouped_availability = availability_df.groupby('number')
station_availability_dict = dict()
for station_id in station_ids: 
    station_availability_dict[station_id] = station_grouped_availability.get_group(station_id)

In [60]:
def bin_feature(df, binned_column_name, column_to_bin, bin_start_values, continuous=False):
    
    # sanity check
    bin_start_values = sorted(bin_start_values)
    
    # creating labels
    bin_names = []
    for bin_start, bin_end in zip(bin_start_values, bin_start_values[1:]):
        if not continuous:
            bin_names.append('{}-{}'.format(bin_start, bin_end))
        else:
            bin_names.append(bin_end)
            
    if not continuous:        
        bin_names.append('{}+'.format(bin_start_values[-1]))
    else:
        bin_names.append(bin_start_values[-1] + (bin_start_values[-1] - bin_start_values[-2]))
    
    def binning_function(value):
        for bin_start, bin_end, bin_label in zip(bin_start_values, bin_start_values[1:], bin_names):
            if value >= bin_start and value < bin_end:
                return bin_label
        # for the last bin
        if value >= bin_start_values[-1]:
            return bin_names[-1]
    
    df[binned_column_name] = df[column_to_bin].apply(binning_function)
    return df

def datetime_to_decimal(fetch_time): 
    hour = fetch_time.hour
    minute = fetch_time.minute
    time_in_decimal = hour + (minute/60)
    return round(time_in_decimal,2)

# Get the day of the week as a string
def datetime_to_day(fetch_time): 
    day_of_week = fetch_time.strftime('%A')
    return day_of_week




In [66]:
#Iterating through the data dictionary to find data of each stations seperately: 
import pickle
model_dict = dict()
for station_id in station_ids: 
    
    df = station_availability_dict[station_id]
    
    max_bikes = df['bike_stands'].iloc[0]

# Modelling subset
    useless_cols = ['id', 'number', 'last_update', 'temp_min', 'temp_max', 'description', 'pressure', 'humidity', 
                   'bike_stands', 'available_bike_stands', 'temp']
    df = df[[col for col in df.columns if col not in useless_cols]]

    # Feature engineering
    
    hour = df['data_fetch_time'].apply(datetime_to_decimal)
    day = df['data_fetch_time'].apply(datetime_to_day)
    df.insert(4, 'hour', hour)
    df.insert(5, 'day_of_week', day)
    
    #Drop time col
    df = df.drop(columns='data_fetch_time')
    
    #custom map of weather from best to worst (subjective)
    weather_ranks = {
        'Clear' : 1,
        'Clouds' : 2,
        'Drizzle' : 3,
        'Mist' : 4,
        'Rain' : 5,
        'Fog' : 6, 
        'Snow' : 7
    }
    
    #day ranks that start at Monday = 1 
    day_ranks = {
        'Monday' : 1,
        'Tuesday' : 2,
        'Wednesday' : 3,
        'Thursday' : 4,
        'Friday' : 5,
        'Saturday' : 6, 
        'Sunday' : 7
    }
    df['main'] = df['main'].map(weather_ranks)
    df['day_of_week'] = df['day_of_week'].map(day_ranks)
    plotting_y = df['available_bikes']
#     # Categorize labels
    df = bin_feature(df, 'available_bikes', 'available_bikes', range(0, max_bikes, 3))

    # X-features, y-labels
    df.rename(columns={'main': 'weather', 'feels_like':'temperature'}, inplace=True)

    maybe_useless = []
    df = df[[col for col in df.columns if col not in maybe_useless]]

    df = df.reset_index(drop=True)
    y = df.pop('available_bikes')
    X = df
    print('features:')
    display(X)
    print('labels:')
    y

    # Creating classifier
    from sklearn.linear_model import LogisticRegression
    clf = KNeighborsClassifier().fit(X, y)
    y_pred = clf.predict(X)
    clf.score(X, y)

    correct_predictions = 0
    for actual, prediction in zip(y, y_pred):
        if actual == prediction:
            correct_predictions += 1
        # print('Actual : {}, Prediction: {}'.format(actual, prediction))
        
    print('The station no:',station_id)
    print('{}/{} predicted accurately'.format(correct_predictions, len(y)))
    station_id_string = str(station_id)
    station_id_string = station_id_string + '.pkl'
    with open(station_id_string, 'wb') as handle:
        pickle.dump(clf, handle, pickle.HIGHEST_PROTOCOL)

features:


,weather,temperature,hour,day_of_week
0,2,3,11.98,5
1,2,3,12.07,5
2,2,4,12.15,5
3,2,4,12.25,5
4,2,4,12.33,5
...,...,...,...,...
8811,2,11,8.78,3
8812,2,11,8.87,3
8813,2,11,8.95,3
8814,2,11,9.03,3


labels:
The station no: 42
7997/8816 predicted accurately
features:


,weather,temperature,hour,day_of_week
0,2,3,11.98,5
1,2,3,12.07,5
2,2,4,12.15,5
3,2,4,12.25,5
4,2,3,12.33,5
...,...,...,...,...
8811,2,11,8.78,3
8812,2,11,8.87,3
8813,2,11,8.95,3
8814,2,11,9.03,3


labels:
The station no: 30
8274/8816 predicted accurately
features:


...

labels:
The station no: 92
7919/8816 predicted accurately
features:


,weather,temperature,hour,day_of_week
0,2,3,11.98,5
1,2,4,12.07,5
2,2,4,12.15,5
3,2,3,12.25,5
4,2,3,12.33,5
...,...,...,...,...
8811,2,11,8.78,3
8812,2,11,8.87,3
8813,2,11,8.95,3
8814,2,11,9.03,3


labels:
The station no: 21
7779/8816 predicted accurately
features:


,weather,temperature,hour,day_of_week
0,2,3,11.98,5
1,2,4,12.07,5
2,2,4,12.15,5
3,2,4,12.25,5
4,2,4,12.33,5
...,...,...,...,...
8811,2,11,8.78,3
8812,2,11,8.87,3
8813,2,11,8.95,3
8814,2,11,9.03,3


labels:
The station no: 88
8009/8816 predicted accurately


In [44]:
y_pred = clf.predict_proba(X)
import numpy as np
for prediction in y_pred:
    top_3_idx = np.argsort(prediction)[-3:]
    print('Top 3 guesses - {}% sure its {}, {}% sure its {}, {}% sure its {}'.format(
    prediction[top_3_idx[-1]]*100, clf.classes_[top_3_idx[-1]],
    prediction[top_3_idx[-2]]*100, clf.classes_[top_3_idx[-2]],
    prediction[top_3_idx[-3]]*100, clf.classes_[top_3_idx[-3]]))

Top 3 guesses - 80.0% sure its 15, 20.0% sure its 30, 0.0% sure its 1
Top 3 guesses - 80.0% sure its 15, 20.0% sure its 30, 0.0% sure its 1
Top 3 guesses - 60.0% sure its 18, 20.0% sure its 19, 20.0% sure its 20
Top 3 guesses - 20.0% sure its 15, 20.0% sure its 28, 20.0% sure its 21
Top 3 guesses - 40.0% sure its 15, 40.0% sure its 13, 20.0% sure its 14
Top 3 guesses - 40.0% sure its 15, 40.0% sure its 13, 20.0% sure its 14
Top 3 guesses - 40.0% sure its 15, 40.0% sure its 13, 20.0% sure its 14
Top 3 guesses - 40.0% sure its 15, 40.0% sure its 13, 20.0% sure its 14
Top 3 guesses - 40.0% sure its 15, 40.0% sure its 13, 20.0% sure its 14
Top 3 guesses - 80.0% sure its 12, 20.0% sure its 13, 0.0% sure its 31
Top 3 guesses - 40.0% sure its 12, 40.0% sure its 26, 20.0% sure its 24
Top 3 guesses - 40.0% sure its 12, 40.0% sure its 26, 20.0% sure its 24
Top 3 guesses - 60.0% sure its 12, 20.0% sure its 28, 20.0% sure its 24
Top 3 guesses - 60.0% sure its 12, 20.0% sure its 28, 20.0% sure its 

In [53]:
# clf.predict()
single = X.head(1)
pred = clf.predict(single)
len(pred)

1

In [7]:
import requests
lat = 0
lon = 0
WEATHER_API_KEY = '084ab11173d072a8e4492921633927de'
weather_api_key=WEATHER_API_KEY
WEATHER_FORECAST_API = "https://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid={" \
                           "weather_api_key}"
weather_forecast_response = requests.get(WEATHER_FORECAST_API.format(lat=lat, lon=lon,
                                                                     weather_api_key=weather_api_key))
print(weather_forecast_response.content.decode())

{"cod":401, "message": "Invalid API key. Please see https://openweathermap.org/faq#error401 for more info."}


In [14]:
endpoint = 'https://api.openweathermap.org/data/3/forecast?lat=44.34&lon=10.99&appid={}'
key = '9f3602b873898a069f03afa3a01b160f'
print('request url: ', endpoint.format(key))
print(requests.get(endpoint.format(key)).content.decode())

request url:  https://api.openweathermap.org/data/3/forecast?lat=44.34&lon=10.99&appid=9f3602b873898a069f03afa3a01b160f
{"cod":401, "message": "Invalid API key. Please see https://openweathermap.org/faq#error401 for more info."}
